## Chapter 7 Exercise 08

Load the MNIST data (introduced in Chapter 3), and split it into a training set, a validation set, and a test set (e.g., use the first 40,000 instances for training, the next 10,000 for validation, and the last 10,000 for testing). Then train various classifiers, such as a Random Forest classifier, an Extra-Trees classifier, and an SVM. Next, try to combine them into an ensemble that outperforms them all on the validation set, using a soft or hard voting classifier. Once you have found one, try it on the test set. How much better does it perform compared to the individual classifiers?

In [103]:
% matplotlib inline
import matplotlib 
import matplotlib.pyplot as plt

import numpy as np

In [104]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
mnist

{'DESCR': 'mldata.org dataset: mnist-original',
 'COL_NAMES': ['label', 'data'],
 'target': array([0., 0., 0., ..., 9., 9., 9.]),
 'data': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)}

In [105]:
X, y = mnist['data'], mnist['target']

In [106]:
# Train:50000, Validation:10000, Test:10000
shuffle_indices = np.random.permutation(60000)
X_train = X[shuffle_indices[:50000]]
y_train = y[shuffle_indices[:50000]]
X_train_mini = X[shuffle_indices[:10000]]
y_train_mini = y[shuffle_indices[:10000]]
X_val = X[shuffle_indices[50000:]]
y_val = y[shuffle_indices[50000:]]
X_test = X[60000:]
y_test = y[60000:]

print('Train:', len(X_train))
print('Train_mini:', len(X_train_mini))
print('Validation:', len(X_val))
print('Test:', len(X_test))

Train: 50000
Train_mini: 10000
Validation: 10000
Test: 10000


In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

rnd_clf = RandomForestClassifier(random_state=42)
ext_clf = ExtraTreesClassifier(random_state=42)
svm_clf = SVC(kernel='linear', probability=True, random_state=42)
vote_clf = VotingClassifier(
    estimators = [('rnd_clf', rnd_clf), ('ext_clf', ext_clf), ('svm_clf', svm_clf)],
    voting = 'hard'
)

In [108]:
from sklearn.metrics import accuracy_score
for clf in (rnd_clf, ext_clf, svm_clf, vote_clf):
    clf.fit(X_train_mini, y_train_mini)
    y_pred = clf.predict(X_val)
    print(clf.__class__.__name__, accuracy_score(y_pred, y_val))

RandomForestClassifier 0.912
ExtraTreesClassifier 0.9201
SVC 0.9124
VotingClassifier 0.9353


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [109]:
soft_vote_clf = VotingClassifier(
    estimators = [('rnd_clf', rnd_clf), ('ext_clf', ext_clf), ('svm_clf', svm_clf)],
    voting = 'soft'
)
soft_vote_clf.fit(X_train_mini, y_train_mini)
y_pred = soft_vote_clf.predict(X_val)
print('Soft-VotingClassifier', accuracy_score(y_pred, y_val))

Soft-VotingClassifier 0.944


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [110]:
# Run on test set
for clf in (rnd_clf, ext_clf, svm_clf, vote_clf, soft_vote_clf):
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_pred, y_test))

RandomForestClassifier 0.9142
ExtraTreesClassifier 0.9204
SVC 0.9168


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


VotingClassifier 0.9379
VotingClassifier 0.948


/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


## Exercise 09
Run the individual classifiers from the previous exercise to make predictions on the validation set, and create a new training set with the resulting predictions: each training instance is a vector containing the set of predictions from all your classifiers for an image, and the target is the image’s class. Congratulations, you have just trained a blender, and together with the classifiers they form a stacking ensemble! Now let’s evaluate the ensemble on the test set. For each image in the test set, make predictions with all your classifiers, then feed the predictions to the blender to get the ensemble’s predictions. How does it compare to the voting clas‐ sifier you trained earlier?

In [111]:
# prepare blender training data
X2_train = []
y2_train = y_val
for clf in (rnd_clf, ext_clf, svm_clf, vote_clf, soft_vote_clf):
    y1_val = clf.predict(X_val)
    X2_train.append(y1_val.tolist())
X2_train = np.transpose(np.array(X2_train))

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [112]:
X2_train.shape

(10000, 5)

In [113]:
# train blender
from sklearn.svm import LinearSVC
blend_clf = SVC()
blend_clf.fit(X2_train, y2_train)

# test
y_pred = blend_clf.predict(X2_train)
print('stack val:', accuracy_score(y_pred, y2_train))

stack val: 0.9514


In [114]:
def stack_predict(X):
    X2 = []
    for clf in (rnd_clf, ext_clf, svm_clf, vote_clf, soft_vote_clf):
        y_pred = clf.predict(X)
        X2.append(y_pred.tolist())
    X2 = np.transpose(np.array(X2))
    return blend_clf.predict(X2)

In [115]:
y_pred = stack_predict(X_test)
print('stack test score:', accuracy_score(y_pred, y_test))

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


stack test score: 0.9409
